<a href="https://colab.research.google.com/github/chabryl/AI-Redirect-Mapping/blob/main/redirect_monitoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Benötigte Pakete installieren
!pip install sentence-transformers qdrant-client pandas

import pandas as pd
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.http import models
import numpy as np
import os
from google.colab import userdata

In [5]:
# Benötigte Pakete installieren
!pip install sentence-transformers qdrant-client pandas

import pandas as pd
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.http import models
import numpy as np
import os
from google.colab import userdata

# 1. Hochladen der CSV-Dateien
from google.colab import files
uploaded = files.upload()

# Dateinamen der hochgeladenen Dateien
crawl_file = 'crawl_all.csv'
error_file = '404_errors.csv'

# 2. CSV-Dateien laden
crawl_data = pd.read_csv(crawl_file)
error_data = pd.read_csv(error_file)

# 3. Word Embeddings nur einmal generieren und speichern
def create_and_save_embeddings(df, save_path, model_name="paraphrase-multilingual-MiniLM-L12-v2"):
    if not os.path.exists(save_path):
        # Lade das vortrainierte Modell
        model = SentenceTransformer(model_name)

        # Erstelle eine Liste der Texte (Title, Meta Description, H1)
        texts = df.apply(lambda row: f"{row['Title']} {row['Meta Description']} {row['H1']}", axis=1).tolist()

        # Generiere Embeddings
        embeddings = model.encode(texts)

        # Speichere die Embeddings und die URLs in eine Datei
        np.savez(save_path, embeddings=embeddings, urls=df['url'].tolist())
        print(f"Embeddings erfolgreich in {save_path} gespeichert.")
    else:
        print(f"Embeddings bereits in {save_path} vorhanden.")

# Erstelle und speichere Embeddings für die crawl_all Datei
embeddings_file = 'crawl_embeddings.npz'
create_and_save_embeddings(crawl_data, embeddings_file, model_name="paraphrase-multilingual-MiniLM-L12-v2")

# 4. Verbindung zur Qdrant-Datenbank
# Hole den API-Key und die API-URL von Qdrant aus den Colab userdata Geheimnissen
api_key_qudrant = userdata.get('api_key_qudrant')
api_url_qudrant = userdata.get('api_url_qudrant')

# Qdrant Client initialisieren
qdrant_client = QdrantClient(url=api_url_qudrant, api_key=api_key_qudrant)

# 5. Embeddings in Qdrant hochladen (falls nicht bereits vorhanden)
def upload_embeddings_to_qdrant(client, collection_name, embeddings_file):
    # Lade die Embeddings und die zugehörigen URLs
    data = np.load(embeddings_file)
    embeddings = data['embeddings']
    urls = data['urls']

    # Überprüfe, ob die Sammlung bereits existiert, und erstelle sie falls nicht
    try:
        client.get_collection(collection_name)
        print(f"Die Sammlung {collection_name} existiert bereits.")
    except:
        print(f"Die Sammlung {collection_name} wird neu erstellt.")
        client.recreate_collection(
            collection_name=collection_name,
            vectors_config=models.VectorParams(
                size=embeddings.shape[1],  # Größe des Embeddings
                distance=models.Distance.COSINE
            )
        )
        client.upload_collection(
            collection_name=collection_name,
            vectors=embeddings,
            payload=[{"url": url} for url in urls]
        )
        print(f"Embeddings erfolgreich in der Sammlung {collection_name} hochgeladen.")

# Qdrant Collection Name
collection_name = "crawl_data_collection"
upload_embeddings_to_qdrant(qdrant_client, collection_name, embeddings_file)

# 6. Ähnlichkeitssuche für 404-Fehler URLs und Speichern der Ergebnisse in CSV-Datei
def find_similar_urls_for_404(client, collection_name, error_data, crawl_data, embeddings_file, model_name="paraphrase-multilingual-MiniLM-L12-v2"):
    # Lade das Modell für die Embedding-Erstellung
    model = SentenceTransformer(model_name)

    # Lade die Embeddings aus der Datei
    data = np.load(embeddings_file)
    embeddings = data['embeddings']
    urls = data['urls']

    # Liste zur Speicherung der Ergebnisse
    results_list = []

    for index, row in error_data.iterrows():
        url_404 = row['url']

        # Suche die Daten zur 404-URL in den Crawldaten
        matching_row = crawl_data[crawl_data['url'] == url_404]

        if matching_row.empty:
            print(f"Keine Daten für die 404-URL {url_404} gefunden")
            continue

        # Erstelle Embedding für die 404-Fehlerseite (basierend auf Title, Meta Description, H1)
        text_404 = f"{matching_row['Title'].values[0]} {matching_row['Meta Description'].values[0]} {matching_row['H1'].values[0]}"
        embedding_404 = model.encode([text_404])[0]

        # Suche ähnliche URLs in der Qdrant Datenbank (aber nicht identische URLs)
        results = client.search(
            collection_name=collection_name,
            query_vector=embedding_404,
            limit=5
        )

        # Sammle die Ergebnisse für die CSV-Ausgabe
        similar_urls = [hit.payload['url'] for hit in results if hit.payload['url'] != url_404]

        # Speichere die 404 URL und die ähnlichen URLs in der Liste
        results_list.append({
            '404_url': url_404,
            'similar_urls': ', '.join(similar_urls)  # Kombiniere die ähnlichen URLs in eine Liste
        })

    # Konvertiere die Ergebnisse in einen DataFrame und speichere sie als CSV
    results_df = pd.DataFrame(results_list)
    results_df.to_csv('similar_urls_for_404.csv', index=False)
    print("Ergebnisse erfolgreich in 'similar_urls_for_404.csv' gespeichert.")

# Suche nach ähnlichen URLs für alle 404 Fehler und speichere die Ergebnisse
find_similar_urls_for_404(qdrant_client, collection_name, error_data, crawl_data, embeddings_file, model_name="paraphrase-multilingual-MiniLM-L12-v2")

# Download der generierten CSV-Datei in Google Colab
from google.colab import files
files.download('similar_urls_for_404.csv')

Saving crawl_all.csv to crawl_all (4).csv
Saving 404_errors.csv to 404_errors (4).csv
Embeddings bereits in crawl_embeddings.npz vorhanden.
Die Sammlung crawl_data_collection existiert bereits.


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Keine Daten für die 404-URL /de/flachdach/absturzsicherung/lichtkuppelsicherung.htmlLichtkuppeldurchsturzsicherung nach DGUV Information 203-058LIGHT-FLEXDie größenvariable kollektive Absturzsicherung LIGHT-FLEX ist die robuste Lösung für verschiedenste Lichtkuppeltypen sowie Lichtbänder. Das System überzeugt durch eine rasche und unkomplizierte Montage und ist auf Holz, Beton, Stahl und Kunststoff möglich.flexible Durchsturzsicherung bei offener und geschlossener Lichtkuppeleinfacher Einbau, einfaches Nachrüstenauch für Rauch-Wärmeabzugsanlagen (RWA)Teilausschnitt für Öffnungsmechanismusvariabel ausführbarZertifizierung nach dem neuesten Stand der Technik:GS-BAU-18, EN 1873:2016, EN 14963:2006 gefunden
Ergebnisse erfolgreich in 'similar_urls_for_404.csv' gespeichert.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>